In [20]:
pip uninstall numpy -y


Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.
You can safely remove it manually.


In [21]:
pip install "numpy>=2.0.0"


  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.2.6 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import numpy
print(numpy.__version__)

2.2.6


In [ ]:
pip install --upgrade numpy

In [2]:
import pickle
import glob
import os

# Path to your folder containing the .pkl files
folder_path = "./graphs"   # or "/absolute/path/to/graphs" if outside your project folder

graphs = []

# Loop through all .pkl files in the folder
for file in glob.glob(os.path.join(folder_path, "*.pkl")):
    with open(file, 'rb') as f:
        graphs.append(pickle.load(f))

print(f"Loaded {len(graphs)} graphs successfully!")


Loaded 112 graphs successfully!


In [3]:
import pickle
import glob
import os
import re
from collections import defaultdict

folder_path = "./graphs"

# Dictionary: teamID → list of graphs
team_graphs = defaultdict(list)

# Regex pattern to extract teamID from filenames
pattern = re.compile(r'team(\d+)\.pkl')

for file in glob.glob(os.path.join(folder_path, "graph_match*_team*.pkl")):
    match = pattern.search(file)
    if match:
        team_id = int(match.group(1))  # convert to integer if needed
        with open(file, 'rb') as f:
            graph = pickle.load(f)
        team_graphs[team_id].append(graph)

print(f"Loaded graphs for {len(team_graphs)} teams:")
for team_id, graphs in team_graphs.items():
    print(f"  Team {team_id}: {len(graphs)} graphs")


Loaded graphs for 3 teams:
  Team 1609: 38 graphs
  Team 1625: 37 graphs
  Team 1612: 37 graphs


In [4]:
# Map numeric team IDs to friendly names
team_name_map = {
    1609: "TeamA",
    1625: "TeamB",
    1612: "TeamC"
}

# Build new dictionary with readable team names
team_graphs_dict = {
    team_name_map[team_id]: graphs
    for team_id, graphs in team_graphs.items()
    if team_id in team_name_map
}

# Quick summary
for name, graphs in team_graphs_dict.items():
    print(f"{name}: {len(graphs)} graphs")


TeamA: 38 graphs
TeamB: 37 graphs
TeamC: 37 graphs


In [5]:
# ===========================================
#  TEAM-LEVEL GCN AUTOENCODER FOR TACTICAL CLUSTERING
# ===========================================
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx
from torch_geometric.loader import DataLoader
import networkx as nx
import numpy as np
from tqdm import tqdm


# -----------------------------
# 1. Prepare each graph as PyG data
# -----------------------------
def prepare_pyg_data(G, zt=None, add_spatial=True):
    """
    Converts a zone transition graph into a PyTorch Geometric Data object.
    Includes node features + edge weights.
    """

    features = []
    for n in G.nodes():
        # Structural features
        in_deg = G.in_degree(n, weight='weight')
        out_deg = G.out_degree(n, weight='weight')
        total_w = in_deg + out_deg

        # Role distribution if available (GK, DF, MD, FW)
        role_dist = G.nodes[n].get("role_distribution", [0, 0, 0, 0])

        # Optional spatial feature
        if zt is not None and add_spatial:
            x_center, y_center = zt.get_zone_center(n)
            x_center, y_center = x_center / 100, y_center / 100
        else:
            x_center, y_center = 0.0, 0.0

        features.append([
            in_deg, out_deg, total_w,
            *role_dist,
            x_center, y_center
        ])

    X = torch.tensor(np.array(features, dtype=float), dtype=torch.float32)

    data = from_networkx(G)
    data.x = X

    # Add edge weights as attributes
    edge_weights = [G[u][v].get("weight", 1.0) for u, v in G.edges()]
    data.edge_attr = torch.tensor(edge_weights, dtype=torch.float32)

    return data


# -----------------------------
# 2. Define the GCN Autoencoder
# -----------------------------
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, latent_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, latent_channels)

    def forward(self, x, edge_index, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight=edge_weight))
        z = self.conv2(x, edge_index, edge_weight=edge_weight)
        return z


class GraphAutoencoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, latent_channels):
        super().__init__()
        self.encoder = GCNEncoder(in_channels, hidden_channels, latent_channels)

    def forward(self, data):
        z = self.encoder(data.x, data.edge_index, data.edge_attr)
        # Adjacency reconstruction (inner product)
        adj_pred = torch.sigmoid(torch.mm(z, z.t()))
        return adj_pred, z


# -----------------------------
# 3. Train on all team graphs jointly
# -----------------------------
def train_team_autoencoder(team_graphs, zt=None, hidden_dim=16, latent_dim=3,
                           epochs=200, lr=0.005, batch_size=1, device="cpu"):

    # Prepare PyG data objects
    data_list = [prepare_pyg_data(G, zt) for G in team_graphs]
    loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)

    # Model setup
    in_dim = data_list[0].x.shape[1]
    model = GraphAutoencoder(in_dim, hidden_dim, latent_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Training loop
    model.train()
    for epoch in tqdm(range(epochs), desc="Training GCN Autoencoder"):
        total_loss = 0
        for data in loader:
            data = data.to(device)
            optimizer.zero_grad()
            A_pred, _ = model(data)

            # Ground-truth adjacency
            A_true = torch.zeros_like(A_pred)
            for i, (u, v) in enumerate(data.edge_index.t()):
                A_true[u, v] = 1.0

            loss = F.mse_loss(A_pred, A_true)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

    return model


# -----------------------------
# 4. Extract embeddings for each match
# -----------------------------
def get_team_embeddings(model, team_graphs, zt=None, device="cpu"):
    model.eval()
    embeddings_per_match = []
    with torch.no_grad():
        for G in team_graphs:
            data = prepare_pyg_data(G, zt).to(device)
            _, z = model(data)
            # Instead of .numpy(), just keep as list
            embeddings_per_match.append(z.cpu().tolist())
    return embeddings_per_match



# -----------------------------
# 5. Example usage
# -----------------------------
# Suppose you have a dict of team graphs:
# team_graphs_dict = {
#     "TeamA": [G_A1, G_A2, ..., G_A40],
#     "TeamB": [G_B1, G_B2, ..., G_B40],
#     "TeamC": [G_C1, G_C2, ..., G_C40],
# }

# Example run for Team A:
# model_A = train_team_autoencoder(team_graphs_dict["TeamA"], zt, epochs=200)
# embeddings_A = get_team_embeddings(model_A, team_graphs_dict["TeamA"], zt)
#
# np.save("teamA_zone_embeddings.npy", embeddings_A)


c:\Users\Kangkanglbk\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Kangkanglbk\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Kangkanglbk\AppData\Local\Programs\Python\Python310\lib\runpy.p

In [6]:
import numpy as np
import pandas as pd

class ZoneTransformer:
    """Simplified version for zone grid + coordinate mapping."""

    def __init__(self):
        # Grid segmentation (8x5 layout)
        self.width_segments = [6, 10, 17, 17, 17, 17, 10, 6]
        self.height_segments = [19, 18, 26, 18, 19]

        self.n_rows = len(self.height_segments)
        self.n_cols = len(self.width_segments)
        self.n_zones = self.n_rows * self.n_cols  # 8 x 5
        self.outside_zone_id = self.n_zones

        self.width_boundaries = np.cumsum([0] + self.width_segments)
        self.height_boundaries = np.cumsum([0] + self.height_segments)
        self.zone_names = self._create_zone_names()

    def _create_zone_names(self):
        row_names = ["RIGHT_WING", "RIGHT_HALF", "CENTER", "LEFT_HALF", "LEFT_WING"]
        col_names = [
            "DEF_BOX",
            "DEF_PENALTY",
            "DEF_THIRD_DEEP",
            "DEF_THIRD",
            "MID_THIRD_DEF",
            "MID_THIRD_ATT",
            "ATT_THIRD",
            "ATT_PENALTY",
        ]
        names = {}
        for col in range(self.n_cols):
            for row in range(self.n_rows):
                zid = self.rowcol_to_id(row, col)
                names[zid] = f"{row_names[row]}_{col_names[col]}"
        names[self.outside_zone_id] = "OUTSIDE"
        return names

    def rowcol_to_id(self, row, col):
        if row < 0 or row >= self.n_rows or col < 0 or col >= self.n_cols:
            return self.outside_zone_id
        return col * self.n_rows + (self.n_rows - row - 1)

    def id_to_rowcol(self, zid):
        if zid == self.outside_zone_id:
            return (-1, -1)
        row = self.n_rows - (zid % self.n_rows) - 1
        col = zid // self.n_rows
        return (row, col)

    def get_zone_bounds(self, zid):
        if zid == self.outside_zone_id:
            return (100.0, 100.0, 100.0, 100.0)
        row, col = self.id_to_rowcol(zid)
        x_min = self.width_boundaries[col]
        x_max = self.width_boundaries[col + 1]
        y_min = self.height_boundaries[row]
        y_max = self.height_boundaries[row + 1]
        return (x_min, x_max, y_min, y_max)

    def get_zone_center(self, zid):
        if zid == self.outside_zone_id:
            return (100.0, 100.0)
        x_min, x_max, y_min, y_max = self.get_zone_bounds(zid)
        return ((x_min + x_max) / 2, (y_min + y_max) / 2)

    def get_zone_name(self, zid):
        return self.zone_names.get(zid, f"ZONE_{zid}")


In [7]:

zt = ZoneTransformer()


In [ ]:
#Models training

model_A = train_team_autoencoder(team_graphs_dict["TeamA"], zt, epochs=200)
model_B = train_team_autoencoder(team_graphs_dict["TeamB"], zt, epochs=200)
model_C = train_team_autoencoder(team_graphs_dict["TeamC"], zt, epochs=200)


Training GCN Autoencoder:  10%|█         | 20/200 [00:14<02:06,  1.42it/s]

Epoch 20/200, Loss: 9.6965


Training GCN Autoencoder:  20%|██        | 40/200 [00:28<01:54,  1.40it/s]

Epoch 40/200, Loss: 9.4895


Training GCN Autoencoder:  30%|███       | 60/200 [00:42<01:38,  1.42it/s]

...

Epoch 160/200, Loss: 8.6803


Training GCN Autoencoder:  90%|█████████ | 180/200 [02:12<00:14,  1.41it/s]

Epoch 180/200, Loss: 8.6492


Training GCN Autoencoder: 100%|██████████| 200/200 [02:27<00:00,  1.36it/s]

Epoch 200/200, Loss: 8.5521


In [ ]:
# KMeans clustering (but only works with entirely with PyTorch because met with problem with NumPy, can also use NumPy directly)

import torch

def kmeans_torch(X, n_clusters=3, n_iters=100, verbose=False):
    """
    Simple KMeans clustering in PyTorch
    X: tensor of shape (n_samples, n_features)
    Returns: cluster assignments, centroids
    """
    n_samples, n_features = X.shape

    # Initialize centroids randomly from data points
    indices = torch.randperm(n_samples)[:n_clusters]
    centroids = X[indices]

    for i in range(n_iters):
        # Compute distances (squared Euclidean)
        dists = torch.cdist(X, centroids, p=2)  # (n_samples, n_clusters)
        # Assign clusters
        cluster_ids = torch.argmin(dists, dim=1)
        # Update centroids
        new_centroids = torch.stack([
            X[cluster_ids == k].mean(dim=0) if (cluster_ids == k).sum() > 0 else centroids[k]
            for k in range(n_clusters)
        ])
        # Check for convergence
        if torch.allclose(centroids, new_centroids):
            break
        centroids = new_centroids
        if verbose:
            print(f"Iteration {i+1} done")
    return cluster_ids, centroids


In [9]:
# Example: get embeddings from trained autoencoders
emb_A = get_team_embeddings(model_A, team_graphs_dict["TeamA"], zt)
emb_B = get_team_embeddings(model_B, team_graphs_dict["TeamB"], zt)
emb_C = get_team_embeddings(model_C, team_graphs_dict["TeamC"], zt)


In [11]:
# Convert your embeddings to a tensor
emb_A_tensor = torch.tensor([item for match in emb_A for item in match], dtype=torch.float32)
emb_B_tensor = torch.tensor([item for match in emb_B for item in match], dtype=torch.float32)
emb_C_tensor = torch.tensor([item for match in emb_C for item in match], dtype=torch.float32)



In [12]:

# Cluster
clusters_A, centroids_A = kmeans_torch(emb_A_tensor, n_clusters=3)
clusters_B, centroids_B = kmeans_torch(emb_B_tensor, n_clusters=3)
clusters_C, centroids_C = kmeans_torch(emb_C_tensor, n_clusters=3)

In [ ]:
# Visualization

import torch
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display  # optional, only for notebook

# -------------------------
# 1) PyTorch KMeans
# -------------------------
def kmeans_torch(X, n_clusters=4, n_iters=200, tol=1e-4, verbose=False, device="cpu"):
    X = X.to(device)
    n_samples, n_features = X.shape
    rand_idx = torch.randperm(n_samples, device=device)[:n_clusters]
    centroids = X[rand_idx].clone()

    for it in range(n_iters):
        dists = torch.cdist(X, centroids, p=2)
        labels = torch.argmin(dists, dim=1)

        new_centroids = torch.zeros_like(centroids)
        converged = True
        for k in range(n_clusters):
            mask = (labels == k)
            if mask.sum() == 0:
                new_centroids[k] = centroids[k]
            else:
                new_centroids[k] = X[mask].mean(dim=0)
            if not torch.allclose(new_centroids[k], centroids[k], atol=tol):
                converged = False
        centroids = new_centroids
        if verbose and (it % 20 == 0 or converged):
            print(f"KMeans iter {it}, converged={converged}")
        if converged:
            break
    return labels.cpu(), centroids.cpu()

# -------------------------
# 2) Pillow visualization
# -------------------------
DEFAULT_COLORS = [
    (31, 119, 180), (255, 127, 14), (44, 160, 44), (214, 39, 40),
    (148, 103, 189), (140, 86, 75), (227, 119, 194), (127, 127, 127)
]

def draw_pitch_zone_clusters(zt, clusters_per_zone, out_path, figsize=(1000, 640), title=None):
    W, H = figsize
    im = Image.new("RGB", (W, H), (26,26,26))
    draw = ImageDraw.Draw(im)
    margin = 30
    def to_px(x, y):
        px = margin + (x / 100.0) * (W - 2 * margin)
        py = margin + ((100 - y) / 100.0) * (H - 2 * margin)
        return px, py

    # pitch border
    draw.rectangle([to_px(0,100), to_px(100,0)], outline=(255,255,255), width=3)

    for zid, cid in enumerate(clusters_per_zone):
        x_min, x_max, y_min, y_max = zt.get_zone_bounds(zid)
        tl = to_px(x_min, y_max)
        br = to_px(x_max, y_min)
        color = DEFAULT_COLORS[cid % len(DEFAULT_COLORS)]
        draw.rectangle([tl, br], fill=color, outline=(255,255,255))
        cx, cy = zt.get_zone_center(zid)
        px, py = to_px(cx, cy)
        try:
            font = ImageFont.truetype("DejaVuSans-Bold.ttf", size=18)
        except:
            font = ImageFont.load_default()
        draw.text((px-10, py-8), str(zid), fill=(255,255,255), font=font)

    if title:
        try:
            font = ImageFont.truetype("DejaVuSans-Bold.ttf", size=22)
        except:
            font = ImageFont.load_default()
        draw.text((W//2 - 200, 10), title, fill=(255,255,255), font=font)

    im.save(out_path)
    return out_path

# -------------------------
# 3) Cluster + draw per team
# -------------------------
def cluster_and_draw_team(embeddings, zt, team_name="Team", n_clusters=4, verbose=False):
    """
    embeddings: list-of-matches, each match = list (n_zones x feature_dim)
    Produces: saved PNG per match and returns majority-vote cluster per zone
    """
    n_matches = len(embeddings)
    all_zone_labels_per_match = []

    for i, match_emb in enumerate(embeddings):
        X = torch.tensor(match_emb, dtype=torch.float32)
        labels, _ = kmeans_torch(X, n_clusters=n_clusters, verbose=verbose)
        zone_labels = labels.tolist()
        all_zone_labels_per_match.append(zone_labels)

        # draw per match
        out_path = f"{team_name}_match{i+1}_clusters.png"
        draw_pitch_zone_clusters(zt, zone_labels, out_path, title=f"{team_name} Match {i+1}")
        if verbose:
            print(f"Saved {out_path}")

    # -------------------------
    # Compute per-zone majority vote across matches
    # -------------------------
    max_zones = max(len(m) for m in embeddings)
    majority_zone_labels = []
    for zid in range(max_zones):
        votes = []
        for match_labels in all_zone_labels_per_match:
            if zid < len(match_labels):
                votes.append(match_labels[zid])
        if votes:
            # majority vote
            vals, counts = torch.unique(torch.tensor(votes), return_counts=True)
            majority = vals[torch.argmax(counts)].item()
        else:
            majority = 0
        majority_zone_labels.append(majority)

    # draw team-level cluster
    team_out_path = f"{team_name}_team_clusters.png"
    draw_pitch_zone_clusters(zt, majority_zone_labels, team_out_path, title=f"{team_name} Majority Clusters")
    if verbose:
        print(f"Saved team-level clusters: {team_out_path}")

    return majority_zone_labels, all_zone_labels_per_match, team_out_path

# -------------------------
# 4) Example usage
# -------------------------
# emb_A, emb_B, emb_C = your previously obtained embeddings
# zt = ZoneTransformer()  # must be defined

# cluster & draw
# majority_A, per_match_A, path_A = cluster_and_draw_team(emb_A, zt, team_name="TeamA", n_clusters=4, verbose=True)
# majority_B, per_match_B, path_B = cluster_and_draw_team(emb_B, zt, team_name="TeamB", n_clusters=4, verbose=True)
# majority_C, per_match_C, path_C = cluster_and_draw_team(emb_C, zt, team_name="TeamC", n_clusters=4, verbose=True)


In [17]:
zt = ZoneTransformer()  # must be defined
majority_A, per_match_A, path_A = cluster_and_draw_team(emb_A, zt, team_name="TeamA", n_clusters=4, verbose=True)
majority_B, per_match_B, path_B = cluster_and_draw_team(emb_B, zt, team_name="TeamB", n_clusters=4, verbose=True)
majority_C, per_match_C, path_C = cluster_and_draw_team(emb_C, zt, team_name="TeamC", n_clusters=4, verbose=True)


KMeans iter 0, converged=False
KMeans iter 3, converged=True
Saved TeamA_match1_clusters.png
KMeans iter 0, converged=False
KMeans iter 4, converged=True
Saved TeamA_match2_clusters.png
KMeans iter 0, converged=False
KMeans iter 8, converged=True
Saved TeamA_match3_clusters.png
KMeans iter 0, converged=False
KMeans iter 2, converged=True
Saved TeamA_match4_clusters.png
KMeans iter 0, converged=False
KMeans iter 6, converged=True
Saved TeamA_match5_clusters.png
KMeans iter 0, converged=False
KMeans iter 10, converged=True
Saved TeamA_match6_clusters.png
KMeans iter 0, converged=False
KMeans iter 4, converged=True
Saved TeamA_match7_clusters.png
KMeans iter 0, converged=False
KMeans iter 2, converged=True
Saved TeamA_match8_clusters.png
KMeans iter 0, converged=False
KMeans iter 8, converged=True
Saved TeamA_match9_clusters.png
KMeans iter 0, converged=False
KMeans iter 2, converged=True
Saved TeamA_match10_clusters.png
KMeans iter 0, converged=False
KMeans iter 1, converged=True
Saved T